<a href="https://colab.research.google.com/github/AnasuyaDutta1412/NLP/blob/main/SMS_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 24.8 MB/s 
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 5.9 MB 49.9 MB/s 
     |████████████████████████████████| 438 kB 61.4 MB/s 
     |████████████████████████████████| 1.7 MB 49.4 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [19]:
import tensorflow_text as text
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [20]:
preprocesses_url="https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
df=pd.read_csv('/content/input_data.csv')
df.head(10)

,Date,Narration,Value Dt,Withdrawal Amt.,Deposit Amt.
0,01/04/21,UPI-K RAJESH-RAJESHDPMINDIA@OKICICI-ICIC000000...,01/04/21,1,NaN
1,01/04/21,"NEFT DR-PUNB0668200-ROHAN-NETBANK, MUM-N091211...",01/04/21,17816,NaN
2,01/04/21,UPI-K RAJESH-RAJESHDPMINDIA@OKICICI-ICIC000000...,01/04/21,24999,NaN
3,01/04/21,UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-1091...,01/04/21,111.18,NaN
4,02/04/21,UPI-MAHENDRAN K M-SRIMURUGANSUPERMARKET1969@OK...,02/04/21,40,NaN
5,02/04/21,UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-1092...,02/04/21,149.98,NaN
6,03/04/21,UPI-MR SHAKSHI NAHAR-SHAKSHI.NAHAR2001@OKHDFCB...,03/04/21,NaN,1
7,04/04/21,UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-1094110...,04/04/21,176,NaN
8,04/04/21,UPI-GOOGLEPAY-GOOG-PAYMENT@OKAXIS-UTIB0000553-...,04/04/21,NaN,5
9,04/04/21,UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-1094...,04/04/21,200,NaN


In [21]:
bert_preprocess_model=hub.KerasLayer(preprocesses_url)

In [34]:
stopwords=list(STOP_WORDS)
stopwords

['see',
 'out',
 'can',
 'am',
 'sixty',
 'made',
 'using',
 'those',
 'of',
 'ten',
 'n’t',
 'since',
 'throughout',
 'seeming',
 'regarding',
 'besides',
 'such',
 'namely',
 'any',
 'how',
 "'re",
 'five',
 'would',
 'eight',
 'its',
 '‘ve',
 'either',
 'might',
 "'m",
 'him',
 'name',
 'third',
 'every',
 'that',
 "'ve",
 'still',
 'towards',
 'last',
 'for',
 'almost',
 'unless',
 'during',
 'just',
 'formerly',
 'front',
 'us',
 'or',
 'on',
 'many',
 'somewhere',
 'four',
 'should',
 'been',
 'other',
 'off',
 'there',
 'elsewhere',
 'doing',
 'must',
 'herein',
 'below',
 'among',
 'between',
 'otherwise',
 'is',
 'the',
 'he',
 'hereby',
 'onto',
 'again',
 'all',
 'by',
 'except',
 'them',
 'when',
 'move',
 'thus',
 'beyond',
 'seems',
 'much',
 'then',
 're',
 'did',
 'via',
 '‘m',
 'down',
 'go',
 'afterwards',
 'meanwhile',
 'thence',
 'full',
 'seem',
 'next',
 'who',
 'before',
 'myself',
 'too',
 'twenty',
 'themselves',
 'wherein',
 'thereby',
 'hereupon',
 'top',
 'h

In [35]:
text_test="""UPI-K RAJESH-RAJESHDPMINDIA@OKICICI-ICIC0000009-109112901059-UPI
NEFT DR-PUNB0668200-ROHAN-NETBANK, MUM-N091211459394676-FUNDS
UPI-K RAJESH-RAJESHDPMINDIA@OKICICI-ICIC0000009-109117511967-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-109117628409-ZOMATO PAYMENT
UPI-MAHENDRAN K M-SRIMURUGANSUPERMARKET1969@OKHDFCBANK-HDFC0001864-109218533308-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-109222438461-ZOMATO PAYMENT
UPI-MR SHAKSHI NAHAR-SHAKSHI.NAHAR2001@OKHDFCBANK-IDIB000G079-109317778469-UPI
UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-109411059391-ZOMATOONLINEORDER
UPI-GOOGLEPAY-GOOG-PAYMENT@OKAXIS-UTIB0000553-109416885748-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-109420825788-ZOMATO PAYMENT
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-109420888945-ZOMATOONLINEORDER
UPI-GOOGLEPAY-GOOG-PAYMENT@OKAXIS-UTIB0000553-109504159090-UPI
UPI-ZERODHA-ZERODHA.RAZORPAY@ICICI-ICIC0000002-109511561213-511005493637809 ZE
UPI-KAPOOR CHAND-AMZN0004251406@APL-PUNB0673800-109621297890-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-109707705376-ZOMATO PAYMENT
UPI-MR SHAKSHI NAHAR-SHAKSHI.NAHAR2001@OKHDFCBANK-IDIB000G079-109718343160-UPI
UPI-ZERODHA-ZERODHA.RAZORPAY@ICICI-ICIC0000002-109810027545-ZERODHA 1055939449
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-109818372806-ZOMATOONLINEORDER
UPI-AMITH KUMAR R-AMITHSHREEMOBILES@OKAXIS-FDRL0001282-109819045664-UPI
POS 435584XXXXXX8149 T NAGAR THIRUMAL
ACH C- IOCL-00000000000009930068
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-109915922589-KITE DEPOSIT 30733
UPI-K  SURYA-SURYAK23467-1@OKAXIS-IOBA0002279-109915319003-UPI
UPI-MAHENDRAN K M-SRIMURUGANSUPERMARKET1969@OKHDFCBANK-HDFC0001864-109917166593-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-110021411504-ZOMATO PAYMENT
UPI-MR JOSEPH KULASEKARA-JOSEPHKULASEKARAN27@OKAXIS-IDIB000M206-110112699305-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-110113385603-ZOMATO PAYMENT
UPI-RAJARAM R-Q93261994@YBL-BKID0008009-110219095820-UPI
UPI-DECATHLON-DECATHLON@YBL-YESB0YBLUPI-110221472779-PAYMENT FOR 701442
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-110316049060-ZOMATO PAYMENT
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-110318233686-ZOMATO PAYMENT
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-110319123728-ZOMATO PAYMENT
UPI-ANIRUDH M-ANIRUDH.MRL@OKHDFCBANK-HDFC0001871-110323157587-UPI
UPI-AKSHAY R-AKSHAY.RADHAVASAN@OKHDFCBANK-HDFC0009095-110323162539-UPI
UPI-MADHAV  GOPAL-MADHAV0908@OKSBI-SBIN0001055-110323046636-MADHAV AND VICHI
UPI-TURF TOWN SPORTING P-EZE0020069@CUB-CIUB0000339-110323166583-UPI
UPI-ADITHYA T-BUNTY.95ADITHYA@OKICICI-KVBL0001213-110400364637-TIKI TAKA
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-110415565706-ZOMATO PAYMENT
UPI-MAHENDRAN K M-SRIMURUGANSUPERMARKET1969@OKHDFCBANK-HDFC0001864-110419857067-UPI
UPI-JOHN BRITTO-7010668784@UPI-SIBL0000110-110511726511-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-110516381422-ZOMATO PAYMENT
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-110610318780-KITE DEPOSIT 29532
UPI-AMAZON SELLER SERVIC-AMAZONSELLERSERVICES.98397377@HDFCBANK-HDFC0000499-110612228376-UPI
UPI-MR KISHORE KUMAR NAH-KISHORE.NAHAR007@OKAXIS-IDIB000G079-110709376564-UPI
UPI-NILANI NALLAMUTHU-NILANI.NALLAMUTHU0901@OKICICI-UBIN0535214-110718676729-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-110812691982-ZOMATO PAYMENT
UPI-MR KISHORE KUMAR NAH-KISHORE.NAHAR007@OKAXIS-IDIB000G079-110909595550-UPI
UPI-MR SHAKSHI NAHAR-SHAKSHI.NAHAR2001@OKHDFCBANK-IDIB000G079-110915425971-UPI
UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-110918954586-ZOMATOONLINEORDER
NWD-435584XXXXXX8149-S1AG0791-CHENNAI
NWD-435584XXXXXX8149-S1AG0791-CHENNAI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-111115476308-ZOMATO PAYMENT
UPI-PAYTM AIRTEL RECHARG-PAYAIR7673@PAYTM-PYTM0123456-111280554678-OID13343571337@PAY
UPI-SHREYA KISHORE NAHAR-SHREYA.NAHAR1998@OKHDFCBANK-HDFC0000082-111215555734-UPI
UPI-MAHALAKSHMI N-MAHA1965LAKSH@OKICICI-KARB0000488-111216917918-UPI
POS 435584XXXXXX8149 T NAGAR THIRUMAL
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-111218239558-ZOMATO PAYMENT
INST-ALERT CHG INC GST JAN-MAR2021-MIR2110910895621
UPI-FLIPKART-FLIPKART.PAYU@HDFCBANK-HDFC0000499-111318192982-UPI TRANSACTION
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-111319288655-ZOMATO PAYMENT
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-111321942912-ZOMATO PAYMENT
UPI-MAHENDRAN K M-SRIMURUGANSUPERMARKET1969@OKHDFCBANK-HDFC0001864-111415419138-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-111417288245-ZOMATO PAYMENT
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-111518999029-SWIGGY ORDER ID 10
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-111607097814-ZOMATO PAYMENT
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-111612684724-ZERODHA EQUITY 687
UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-111617883977-ZOMATOONLINEORDER
NEFT DR-PUNB0668200-ROHAN-NETBANK, MUM-N117211485790547-FUNDS
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-111715353032-ZOMATO PAYMENT
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-111807924757-ZOMATO PAYMENT
UPI-MAHENDRAN K M-SRIMURUGANSUPERMARKET1969@OKHDFCBANK-HDFC0001864-111819039862-THE
UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-111907025188-ZOMATOONLINEORDER
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-111912616791-ZERODHA EQUITY 954
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-111914441455-ZOMATOONLINEORDER
UPI-MCRENNETT  HABIBULLA-Q83032856@YBL-YESB0YBLUPI-112020946065-UPI
UPI-RAJAMMA-Q77223686@YBL-FDRL0001386-112122736133-THE
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-112208723651-ZOMATO PAYMENT
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-112220978804-ZOMATOONLINEORDER
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-112313144985-SWIGGY ORDER ID 10
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-112317057317-ZOMATO PAYMENT
UPI-KAPOOR CHAND-AMZN0004251406@APL-PUNB0673800-112317223124-UPI
UPI-BHARATPEMERCHANT-BHARATPE.9042524061@ICICI-ICIC0000001-112317232898-PAY TO BROWN TREE
UPI-BHARATPEMERCHANT-BHARATPE.9042524061@ICICI-ICIC0000001-112317244756-PAY TO BROWN TREE
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-112321003522-SWIGGY ORDER ID 10
IB FUNDS TRANSFER DR-50100416578348  -SHREYANS NAHAR
UPI-NAGARAJ  T-Q17451014@YBL-SBIN0014203-112417915348-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-112418319250-ZOMATO PAYMENT
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-112507696460-ZOMATOONLINEORDER
UPI-MR KISHORE KUMAR NAH-KISHORE.NAHAR007@OKAXIS-IDIB000G079-112610362024-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-112718078039-ZOMATO PAYMENT
UPI-RPANNEERSELVAM-PANNERSELVAM76@OKAXIS-LAVB0000431-112813959062-UPI
POS 435584XXXXXX8149 JUST BUY CYCLES
UPI-FLIPKART INTERNET PV-FLIPKART1@INDUS-INDB0002201-112819818722-UPI TRANSACTION
UPI-DURGA PRASAD  UPADHY-DURGAPRASADUPADHYAY18-3@OKSBI-SBIN0018393-112820403901-UPI
UPI-DURGA PRASAD  UPADHY-DURGAPRASADUPADHYAY18-3@OKSBI-SBIN0018393-112918363157-UPI
UPI-BALAJI SANDWICH EGMO-PAYTMQR281005050101GUR1JA28IFDE@PAYTM-PYTM0123456-112918685161-UPI
UPI-S DIVYA-PAYTMQR2810050501011K7JHXYYIKIZ@PAYTM-PYTM0123456-112918711353-UPI
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-112920492423-ZOMATOONLINEORDER
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-112920071185-PAYMENT FOR SWIGGY
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-112921452885-TXN FAILED AT MERC
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-113009948332-ZERODHA EQUITY 756
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-113019216082-ZOMATO PAYMENT
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-113020458017-SWIGGY ORDER ID 10
UPI-MR KISHORE KUMAR NAH-KISHORE.NAHAR007@OKAXIS-IDIB000G079-113111143059-UPI
UPI-MR SHAKSHI NAHAR-SHAKSHI.NAHAR2001@OKHDFCBANK-IDIB000G079-113111154092-UPI
UPI-ZERODHA-ZERODHA.RAZORPAY@ICICI-ICIC0000002-113112275233-ZERODHA 4148431785
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-113120456418-ZOMATOONLINEORDER
UPI-SHREYA KISHORE NAHAR-SHREYA.NAHAR1998@OKHDFCBANK-HDFC0000082-113213073499-UPI
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-113319703013-SWIGGY ORDER ID 10
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-113319549008-ZOMATOONLINEORDER
UPI-MR KISHORE KUMAR NAH-KISHORE.NAHAR007@OKAXIS-IDIB000G079-113418933288-UPI
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-113514683643-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-113520496024-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-113614925896-SWIGGY ORDER ID 10
UPI-PAYTM AIRTEL RECHARG-PAYAIR7673@PAYTM-PYTM0123456-113685982323-OID13596632585@PAY
UPI-FLIPKART-FLIPKART.PAYU@HDFCBANK-HDFC0000499-113702714361-UPI TRANSACTION
UPI-COFFEE HOUSE AT MOUN-PAYTM-52005529@PAYTM-PYTM0123456-113818354943-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-113818722570-ZOMATO PAYMENT
UPI-DAYASHANKAR  DUBEY-Q07099181@YBL-KKBK0008487-113921850483-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-114008157293-ZOMATO PAYMENT
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-114023538226-UPI
NEFT CR-SBIN0050661-MR  ASHISH  G-SHREYANS NAHAR-SBIN421141947356
UPI-BILLDESKTEZ-BILLDESK.ELECTRICITY@ICICI-ICIC0000555-114111018698-UPI
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-114118272856-ZOMATOONLINEORDER
UPI-DAYASHANKAR  DUBEY-Q07099181@YBL-KKBK0008487-114120756305-UPI
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-114212153682-PAYMENT FOR SWIGGY
UPI-DURGA PRASAD  UPADHY-DURGAPRASADUPADHYAY18-3@OKSBI-SBIN0018393-114216368009-UPI
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-114218385460-PAYMENT FOR SWIGGY
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-114218255321-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-114220916871-SWIGGY ORDER ID 10
UPI-COFFEE HOUSE AT MOUN-PAYTM-52005529@PAYTM-PYTM0123456-114311078395-UPI
50100327843595-TPT-CHECK-MANJULA NAHAR
UPI-GKM FURNITURES-GPAY-11166927578@OKBIZAXIS-UTIB0000000-114316002455-UPI
UPI-MR KISHORE KUMAR NAH-KISHORE.NAHAR007@OKAXIS-IDIB000G079-114411945591-UPI
UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-114418358944-ZOMATOONLINEORDER
UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-114419428117-ZOMATOONLINEORDER
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-114509771820-UPI
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-114610337853-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-114707688807-ZOMATO PAYMENT
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-114718262034-PAYMENT FOR SWIGGY
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-114719446449-PAYMENT FOR SWIGGY
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-114809216765-PAYMENT FOR SWIGGY
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-114818097587-SWIGGY ORDER ID 10
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-114818894697-ZOMATO PAYMENT
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-114918163326-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-114918392120-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGY.REFUNDS@AXISBANK-UTIB0000052-114918032396-REFUND FROM SWIGGY
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-114918316306-ZOMATO PAYMENT
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-115013903923-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-115020215737-SWIGGY ORDER ID 10
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-115107331325-ZOMATOONLINEORDER
NEFT CR-INDB0000006-WESTERN UNION INTERNATIONAL BANK G-SHREYANS NAHAR-000346477566 PURPOSE OF PAYMENT PAYROLL WA
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-115212581973-ZOMATO PAYMENT
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-115213195611-SWIGGY ORDER ID 10
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-115213854831-ZOMATO PAYMENT
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-115218068917-ZOMATO PAYMENT
UPI-RPANNEERSELVAM-PANNERSELVAM76@OKAXIS-LAVB0000431-115313236395-UPI
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-115320009338-PAYMENT FOR SWIGGY
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-115320473225-SWIGGY ORDER ID 10
IB FUNDS TRANSFER DR-50100416578348  -SHREYANS NAHAR
UPI-SATHISH KUMAR S-JOSEPHSATHISH1982@OKHDFCBANK-UBIN0561177-115407819707-TAKE CARE
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-115418578680-ZOMATO PAYMENT
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-115520474601-ZOMATOONLINEORDER
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-115713583387-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-115719881540-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-115720355522-SWIGGY ORDER ID 10
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-115808155887-ZOMATO PAYMENT
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-115818676015-PAYMENT FOR SWIGGY
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-115913633211-SWIGGY ORDER ID 10
UPI-PAYTM AIRTEL RECHARG-PAYAIR7673@PAYTM-PYTM0123456-115973124544-OID13854562522@PAY
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-116008043839-ZOMATO PAYMENT
UPI-ZERODHA BROKING LTD-ZERODHABROKING@HDFCBANK-HDFC0000523-116011068143-UPI
UPI-SHREYA KISHORE NAHAR-SHREYA.NAHAR1998@OKHDFCBANK-HDFC0000082-116017000989-UPI
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-116108401516-ZOMATO PAYMENT
UPI-FRUIT SHOP GOPALAPUR-PAYTMQR2810050501011S26H6RGEO5H@PAYTM-PYTM0123456-116120601010-UPI
UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-116209082183-ZOMATOONLINEORDER
POS 435584XXXXXX8149 SWIGGY
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-116219173301-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-116219393583-SWIGGY ORDER ID 10
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-116261869115-PAYMENT FOR SWIGGY
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-116312823074-SWIGGY ORDER ID 10
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-116367440566-PAYMENT FOR SWIGGY
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-116412844071-SWIGGY ORDER ID 10
UPI-KHAJA MOHIDEEN  I-ISMAILKHAJA9@OKAXIS-IOBA0002051-116416933157-UPI
UPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-116418151473-ZOMATOONLINEORDER
UPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-116508284974-ZOMATOONLINEORDER
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-116513693773-SWIGGY ORDER ID 10
UPI-EZILAGAM-PAYTMQR2810050501011RQMIFRWFGG2@PAYTM-PYTM0123456-116518370988-UPI
UPI-AAVIN-PAYTMQR281005050101IP90RXRQIQU4@PAYTM-PYTM0123456-116519826662-UPI
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-116612521981-SWIGGY ORDER ID 10
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-116600814704-PAYMENT FOR SWIGGY
UPI-URBAN COMPANY-PAYTM-URBANCLAP@PAYTM-PYTM0123456-116710500921-UPI INTENT
IB FUNDS TRANSFER CR-50100416578348  -SHREYANS NAHAR
UPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-116753212570-EXPRESS
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-116707459853-PAYMENT FOR SWIGGY
UPI-FLIPKART-FLIPKART.PAYU@HDFCBANK-HDFC0000499-116718039989-UPI TRANSACTION
UPI-FLIPKART-FLIPKART.PAYU@HDFCBANK-HDFC0000499-116718099546-UPI TRANSACTION
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-116814204226-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-116819624619-SWIGGY ORDER ID 10
UPI-MR KISHORE KUMAR NAH-KISHORE.NAHAR007@OKAXIS-IDIB000G079-116914767107-UPI
UPI-COFFEE HOUSE AT MOUN-PAYTM-52005529@PAYTM-PYTM0123456-116921600777-UPI
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-117208535926-SWIGGY ORDER ID 10
UPI-NAGARAJ  T-Q17451014@YBL-SBIN0014203-117246833033-UPI
UPI-NAGARAJ  T-Q17451014@YBL-SBIN0014203-117246930865-UPI
UPI-SWIGGY-UPISWIGGY@ICICI-ICIC0000001-117591023449-PAYMENT FOR SWIGGY
UPI-ASWINI AUTOMOBILES-PAYTMQR281005050101E0EDX2XBG6WY@PAYTM-PYTM0123456-117579817468-UPI
ATW-435584XXXXXX8149-S1ACCN64-CHENNAI
ATW-435584XXXXXX8149-S1ACCN64-CHENNAI
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-117612805420-SWIGGY ORDER ID 10
IB FUNDS TRANSFER CR-50100416578348  -SHREYANS NAHAR
UPI-NAGARAJ  T-Q17451014@YBL-SBIN0014203-117687015040-UPI
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-117714179434-SWIGGY ORDER ID 10
UPI-COFFEE HOUSE AT MOUN-PAYTM-52005529@PAYTM-PYTM0123456-117798559179-UPI
UPI-NAGARAJ  T-Q17451014@YBL-SBIN0014203-117802049416-UPI
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-117809426797-SWIGGY ORDER ID 10
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-117813984095-SWIGGY ORDER ID 10
NEFT CR-INDB0000006-WESTERN UNION INTERNATIONAL BANK G-SHREYANS NAHAR-000351664985 PURPOSE OF PAYMENT PAYROLL WA
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-117917210429-SWIGGY ORDER ID 10
UPI-MANJULA NAHAR-MANJULA.NAHAR1972@OKHDFCBANK-HDFC0002043-118019714667-UPI
UPI-NAGARAJ  T-Q17451014@YBL-SBIN0014203-118023554411-UPI
UPI-SWIGGY-SWIGGYUPI@AXISBANK-UTIB0000000-118014672621-SWIGGY ORDER ID 10
UPI-MR SHAKSHI NAHAR-SHAKSHI.NAHAR2001@OKHDFCBANK-IDIB000G079-118132476706-UPI
IB FUNDS TRANSFER DR-50100416578348  -SHREYANS NAHAR
NEFT DR-PUNB0668200-ROHAN-NETBANK, MUM-N181211549454535-FUNDS
"""
text_test

'UPI-K RAJESH-RAJESHDPMINDIA@OKICICI-ICIC0000009-109112901059-UPI\nNEFT DR-PUNB0668200-ROHAN-NETBANK, MUM-N091211459394676-FUNDS\nUPI-K RAJESH-RAJESHDPMINDIA@OKICICI-ICIC0000009-109117511967-UPI\nUPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-109117628409-ZOMATO PAYMENT\nUPI-MAHENDRAN K M-SRIMURUGANSUPERMARKET1969@OKHDFCBANK-HDFC0001864-109218533308-UPI\nUPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-109222438461-ZOMATO PAYMENT\nUPI-MR SHAKSHI NAHAR-SHAKSHI.NAHAR2001@OKHDFCBANK-IDIB000G079-109317778469-UPI\nUPI-ZOMATO-ZOMATO@HDFCBANK-HDFC0000499-109411059391-ZOMATOONLINEORDER\nUPI-GOOGLEPAY-GOOG-PAYMENT@OKAXIS-UTIB0000553-109416885748-UPI\nUPI-ZOMATO-ZOMATO-ORDER@PAYTM-PYTM0123456-109420825788-ZOMATO PAYMENT\nUPI-RAZORPAYZOMATO-ZOMATOINDIA@ICICI-ICIC0000001-109420888945-ZOMATOONLINEORDER\nUPI-GOOGLEPAY-GOOG-PAYMENT@OKAXIS-UTIB0000553-109504159090-UPI\nUPI-ZERODHA-ZERODHA.RAZORPAY@ICICI-ICIC0000002-109511561213-511005493637809 ZE\nUPI-KAPOOR CHAND-AMZN0004251406@APL-PUNB0673800-109621297890-UPI\n

In [36]:
nlp=spacy.load('en_core_web_sm')
nlp

In [43]:
doc=nlp(text_test)   

In [46]:
text = text_test.split("\n")
tokens = [i.split("-") for i in text]
print(tokens)
# tokens=[token.text for token in doc]
# print(tokens)

[['UPI', 'K RAJESH', 'RAJESHDPMINDIA@OKICICI', 'ICIC0000009', '109112901059', 'UPI'], ['NEFT DR', 'PUNB0668200', 'ROHAN', 'NETBANK, MUM', 'N091211459394676', 'FUNDS'], ['UPI', 'K RAJESH', 'RAJESHDPMINDIA@OKICICI', 'ICIC0000009', '109117511967', 'UPI'], ['UPI', 'ZOMATO', 'ZOMATO', 'ORDER@PAYTM', 'PYTM0123456', '109117628409', 'ZOMATO PAYMENT'], ['UPI', 'MAHENDRAN K M', 'SRIMURUGANSUPERMARKET1969@OKHDFCBANK', 'HDFC0001864', '109218533308', 'UPI'], ['UPI', 'ZOMATO', 'ZOMATO', 'ORDER@PAYTM', 'PYTM0123456', '109222438461', 'ZOMATO PAYMENT'], ['UPI', 'MR SHAKSHI NAHAR', 'SHAKSHI.NAHAR2001@OKHDFCBANK', 'IDIB000G079', '109317778469', 'UPI'], ['UPI', 'ZOMATO', 'ZOMATO@HDFCBANK', 'HDFC0000499', '109411059391', 'ZOMATOONLINEORDER'], ['UPI', 'GOOGLEPAY', 'GOOG', 'PAYMENT@OKAXIS', 'UTIB0000553', '109416885748', 'UPI'], ['UPI', 'ZOMATO', 'ZOMATO', 'ORDER@PAYTM', 'PYTM0123456', '109420825788', 'ZOMATO PAYMENT'], ['UPI', 'RAZORPAYZOMATO', 'ZOMATOINDIA@ICICI', 'ICIC0000001', '109420888945', 'ZOMATOONLI

In [58]:
for i in tokens:
    if(len(i)>2):
      a = i[0]
      b = i[1]
    a = i[0]
    print(a,b)


UPI K RAJESH
NEFT DR PUNB0668200
UPI K RAJESH
UPI ZOMATO
UPI MAHENDRAN K M
UPI ZOMATO
UPI MR SHAKSHI NAHAR
UPI ZOMATO
UPI GOOGLEPAY
UPI ZOMATO
UPI RAZORPAYZOMATO
UPI GOOGLEPAY
UPI ZERODHA
UPI KAPOOR CHAND
UPI ZOMATO
UPI MR SHAKSHI NAHAR
UPI ZERODHA
UPI RAZORPAYZOMATO
UPI AMITH KUMAR R
POS 435584XXXXXX8149 T NAGAR THIRUMAL AMITH KUMAR R
ACH C  IOCL
UPI ZERODHA BROKING LTD
UPI K  SURYA
UPI MAHENDRAN K M
UPI ZOMATO
UPI MR JOSEPH KULASEKARA
UPI ZOMATO
UPI RAJARAM R
UPI DECATHLON
UPI ZOMATO
UPI ZOMATO
UPI ZOMATO
UPI ANIRUDH M
UPI AKSHAY R
UPI MADHAV  GOPAL
UPI TURF TOWN SPORTING P
UPI ADITHYA T
UPI ZOMATO
UPI MAHENDRAN K M
UPI JOHN BRITTO
UPI ZOMATO
UPI ZERODHA BROKING LTD
UPI AMAZON SELLER SERVIC
UPI MR KISHORE KUMAR NAH
UPI NILANI NALLAMUTHU
UPI ZOMATO
UPI MR KISHORE KUMAR NAH
UPI MR SHAKSHI NAHAR
UPI ZOMATO
NWD 435584XXXXXX8149
NWD 435584XXXXXX8149
UPI ZOMATO
UPI PAYTM AIRTEL RECHARG
UPI SHREYA KISHORE NAHAR
UPI MAHALAKSHMI N
POS 435584XXXXXX8149 T NAGAR THIRUMAL MAHALAKSHMI N
UPI ZOMATO